In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Parameters

In [3]:
# Fixed parameters

DEVICE = 'ibm_sherbrooke'
_RESETS = True

backend = provider.get_backend(DEVICE)

In [4]:
# Variable parameters
import numpy as np

logicals = ['0', '1']
x_basis = [True, False]

rounds = [10, 20, 30, 40, 50, 75, 100]


print("rounds:", rounds)


rounds: [10, 20, 30, 40, 50, 75, 100]


# Find the best layout

In [5]:
from soft_info import BackendEvaluator

evaluator = BackendEvaluator(backend)
longest_path, _, _, path_info = evaluator.find_longest_good_RepCode_string(plot=False, Ancilla_threshold=0.2)

Trying RepCode string of length 109 => distance 55... Current maxs: CX: 1.0, Ancilla: 1.0
Trying RepCode string of length 107 => distance 54... Current maxs: CX: 1.0, Ancilla: 0.4728
Trying RepCode string of length 105 => distance 53... Current maxs: CX: 1.0, Ancilla: 0.07666666666666666
Trying RepCode string of length 103 => distance 52... Current maxs: CX: 0.021754282174994144, Ancilla: 0.4728
Trying RepCode string of length 101 => distance 51... Current maxs: CX: 0.023407315771999254, Ancilla: 0.25749999999999995


In [6]:
path_info

{'mean_gate_error': 0.008371841209949916,
 'min_gate_error': 0.003290671973908671,
 'max_gate_error': 0.021754282174994144,
 'mean_readout_error': 0.0194069306930693,
 'min_readout_error': 0.0026999999999999247,
 'max_readout_error': 0.25749999999999995,
 'mean_ancilla_error': 0.014985999999999985,
 'min_ancilla_error': 0.0033999999999999586,
 'max_ancilla_error': 0.06309999999999993,
 'CX_fidelity': 0.4310732434290401}

In [7]:
"0.45279818562360047}"

link_qubits = longest_path[1::2]
code_qubits = longest_path[0::2]
layout = link_qubits + code_qubits
distance = len(code_qubits)
print("longest path:", longest_path)
print("layout:", layout)
print("distance:", distance) 

longest path: [19, 18, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17, 30, 31, 32, 36, 51, 50, 49, 55, 68, 69, 70, 74, 89, 88, 87, 93, 106, 107, 108, 112, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 109, 96, 95, 94, 90, 75, 76, 77, 71, 58, 59, 60, 61, 62, 72, 81, 80, 79, 91, 98, 99, 100, 101, 102, 92, 83, 84, 85, 73, 66, 65, 64, 54, 45, 46, 47, 35, 28, 27, 26, 25, 24, 34, 43, 42, 41, 40, 39, 38, 37, 52]
layout: [18, 0, 2, 4, 6, 8, 10, 12, 30, 32, 51, 49, 68, 70, 89, 87, 106, 108, 126, 124, 122, 120, 118, 116, 114, 96, 94, 75, 77, 58, 60, 62, 81, 79, 98, 100, 102, 83, 85, 66, 64, 45, 47, 28, 26, 24, 43, 41, 39, 37, 19, 14, 1, 3, 5, 7, 9, 11, 17, 31, 36, 50, 55, 69, 74, 88, 93, 107, 112, 125, 123, 121, 119, 117, 115, 109, 95, 90, 76, 71, 59, 61, 72, 80, 91, 99, 101, 92, 84, 73, 65, 54, 46, 35, 27, 25, 34, 42, 40, 38, 52]
distance: 51


# No rounds loop

In [9]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import run_IQ_calibration
from qiskit import transpile
from Scratch import metadata_helper
from tqdm import tqdm


############################

# Round index
round_index = 4

# Z = 0
xbasis = False
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20
# REPETITIONS = 1

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
multiplier = 240
# qc.delay(unit='s', duration=120*range_rd[1])
qc.delay(unit='s', duration=multiplier*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=0) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)



# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', rep_delay=range_rd[1],
                job_tags=[f'{multiplier} del shots{n_shots}  {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)


Submitting for rounds =  50


100%|██████████| 20/20 [03:04<00:00,  9.22s/it]


In [ ]:
############################

# Z = 1
xbasis = False
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=multiplier*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=0) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', rep_delay=range_rd[1],
                job_tags=[f'{multiplier} del shots{n_shots}  {distance}, {T}, {logical}, xbasis={xbasis}'])
        
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  10


100%|██████████| 20/20 [00:58<00:00,  2.93s/it]


In [ ]:
############################

# X = 0
xbasis = True
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=multiplier*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=0) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', rep_delay=range_rd[1],
                job_tags=[f'{multiplier} del shots{n_shots}  {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  10


100%|██████████| 20/20 [01:08<00:00,  3.42s/it]


In [ ]:
############################

# X = 1
xbasis = True
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=multiplier*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=0) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', rep_delay=range_rd[1],
                job_tags=[f'{multiplier} del shots{n_shots}  {distance}, {T}, {logical}, xbasis={xbasis}'])
        
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  10


100%|██████████| 20/20 [01:01<00:00,  3.09s/it]
